# Minimal Workflow in Jupyter

This notebook demonstrates how to train models and run HMC from within a Jupyter notebook.
It does not cover loading/saving models.

In [ ]:
import pandas as pd

from nflows_xy.autocorr import autocorrelations
from nflows_xy.core import FlowBasedSampler, PullbackAction
from nflows_xy.flows import HierarchicalFlow
from nflows_xy.hmc import hmc
from nflows_xy.train import train, test
from nflows_xy.xy import action, top_charge, spin_correlation

## 1. Training

In [ ]:
L = 16
β = 5.0

flow = HierarchicalFlow(
    lattice_size=L,
    n_mixture=12,
    weighted=False,
    net_shape=[12],
    net_activation="Tanh",
)
target = action(beta=β, lattice_size=L, lattice_dim=1)
model = FlowBasedSampler(flow, target)

_ = model(1)

model

In [ ]:
model = model.to("cuda")
training_metrics = train(
    model,
    n_steps=2000,
    batch_size=2048,
)
model = model.to("cpu")

_ = training_metrics.plot(
    x="step",
    y=["loss", "ess"],
    subplots=True,
    linestyle="--",
    marker=".",
)

In [ ]:
training_metrics["one_minus_ess"] = 1 - training_metrics.ess
training_metrics.plot(
    x="step",
    y=["one_minus_ess", "vlw"],
    subplots=True,
    linestyle="--",
    marker=".",
    logy=True,
)

In [ ]:
test_metrics = test(model, batch_size=4096)

test_metrics.describe()

In [ ]:
test_metrics.ess.hist(bins=15) 

## 2. Hybrid Monte Carlo

In [ ]:
hmc?

In [ ]:
φ, hmc_metrics = hmc(
    action=target,
    n_replica=64,
    n_traj=10000,
    step_size=0.1,
    traj_length=1.0,
)

print(φ.shape, hmc_metrics)

In [ ]:
Q = top_charge(φ)

df = pd.DataFrame(data=Q.squeeze(-1).T.numpy(), columns=[f"r{i}" for i in range(1, len(Q) + 1)])
ax = df[["r1", "r2"]].plot(xlabel="Monte Carlo time", ylabel="Q")
#ax2 = ax.twiny()
#df.mean(axis=0).hist(bins=16, orientation="horizontal", density=True, ax=ax2)

## 3. Flow-HMC

In [ ]:
φ2, fhmc_metrics = hmc(
    action=PullbackAction(model.flow, model.target),
    n_replica=64,
    n_traj=1000,
    step_size=0.1,
    traj_length=1.0,
)
print(φ2.shape, fhmc_metrics)

In [ ]:
Q2 = top_charge(φ2)

print(Q2.shape)

import pandas as pd
df2 = pd.DataFrame(data=Q2.squeeze(-1).T.numpy(), columns=[f"r{i}" for i in range(1, len(Q2) + 1)])
ax = df2[["r1", "r2"]].plot(xlabel="Monte Carlo time", ylabel="Q")
#ax2 = ax.twiny()
#df2.mean(axis=0).hist(bins=16, orientation="horizontal", histtype="step", ax=ax2)

# Running Scripts from within Jupyter

You can also just run the command line scripts directly from within a Jupyter notebook by prepending the command with `!`, or by using `%%bash` to execute an entire cell with bash.

In [ ]:
!nfxy --help

In [ ]:
%%bash

echo "Running HMC benchmark in ${SHELL}"
nfxy hmc-benchmark -c ../examples/benchmark.yaml

## 1. Training

In [ ]:
!nfxy train --help